Задание **Lite**

Выполните задания, по каждому варианту напишите точность распознавания на проверочной выборке и сделайте выводы (тексты писателей).

А. Запустите нейронку c bag of words (01) при разных maxWordsCount

1. 100
2. 1000
3. 10000
4. 50000

Б. Запустите нейронку c bag of words (01) при maxWordsCount = 20000 и разных архитектурах

1. Поменяйте количество нейронов в слоях
2. Поменяйте количество слоев
3. Поменяйте активационные функции слоев

В. Запустите нейронку c Embbedding при maxWordsCount = 50000, поменяйте размер Embedding пространства

1. 10
2. 50
3. 200

# Подключение библиотек 

In [ ]:
# Для начала работы подключим все необходимые библиотеки
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, SpatialDropout1D, BatchNormalization, Flatten, Embedding, Activation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import utils
from google.colab import drive
import numpy as np
import os
import time
from google.colab import files
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline


# Объявим все необходимые функции

In [ ]:
# Создадим функцию для первоначальной обработки файла
def readText(fileName):
  op = open(fileName, 'r')        # Указываем что открытие будет в режиме чтения
  text = op.read()                # Читаем файл с текстом
  text = text.replace("\n", " ")  # Заменям переносы строк на пробелы
  return text                     # Возвращаем измененный файл

In [ ]:
# Для начала создадим функции. 
# Первая для разделения на короткие векторы с заданным шагом и размером

def getSetFromIndexes (wordIndexes, xLen, step):     # В качестве аргументов принимает (trainWordIndexes или testWordIndexes - список, размер и шаг) 
  xSample = []                                        # Созадим пустой спискок для хранения векторов
  wordsLen = len(wordIndexes)                         # Кол-во слов в поданных на вход trainWordIndexes или testWordIndexes
  index = 0                                           # Начальный индекс (с помощью этой переменной будем сдвигать шаг)


  while (index + xLen <= wordsLen):                   # Задаем цикл пока не дойдем до конца    
    xSample.append(wordIndexes[index:index + xLen])   # На каждой итерации добавляем в список кусок вектора заданной длины
    index += step                                     # Смещаемся на шаг

  return xSample

In [ ]:
# Вторая функция для формирования обучающей и проверочной выборок

def createSetsMultiClasses(wordIndexes, xLen, step):
  nClasses = len(wordIndexes)                            # Зададим кол-во классов выборки
  classesXSample = []                                    # Создадим пустой список в котором будут храниться список нарезанных кусочков предыдущей функцией getSetFromIndexes

  # Пройдемся циклом по каждому тексту выборки из последовательнстей индексов
  for i in wordIndexes:
    classesXSample.append(getSetFromIndexes(i, xLen, step))
  
  # Сформируем один общий список
  xSamples = []                                           # Создаем пустые списки
  ySamples = []

  for t in range(nClasses):                               # В нашем случаи цикл от 0 до 6 (кол-во классов)
    xT = classesXSample[t]                                # На каждой итерации цикла берем очередной нарезанный текст одного из класса

    for i in range (len(xT)):                             # Циклом проходимся по каждому его окну
      xSamples.append(xT[i])                              # Добавляем в общий список обучающей выборке
      ySamples.append(utils.to_categorical(t, nClasses))  # И соответствующий вектор класса
  
  # Переводим в numpy массив
  xSamples = np.array(xSamples)                           
  ySamples = np.array(ySamples)    

  # B возвращаем выборки
  return (xSamples, ySamples)

In [ ]:
# Создадим функцию которая будет представлять выборку в удобном для распознавания размерах

def createTestMultiClasses(wordIndexes, xLen, step):

  # Создаем тестовую выборку из индексов для каждого из 6 классов
  nClasses = len(wordIndexes)                                             # Переменной nClasses присвоим кол-во классов
  xTest6ClassesBow = []                                                   # Создадим пустой список, в котором будут список из всех классов
  xTest6Classec = []                                                      # Создадим список массивов

  # Пройдемся циклом по каждому тестовому тексту из последовательности индексов
  for i in wordIndexes:
    sample = (getSetFromIndexes(i,xLen, step))                            # Воспользуемся ранее написанной функцией для нарезки на векторы
    xTest6Classec.append(sample)                                          # Добавляем в список на каждой итерации 
    xTest6ClassesBow.append(tokenizer.sequences_to_matrix(sample))        # Трансформируем в BOW
  
  xTest6ClassesBow = np.array(xTest6ClassesBow)                           # Переводим в numpy массив
  xTest6Classec = np.array(xTest6Classec)                                 # Переводим в numpy массив

  return xTest6ClassesBow, xTest6Classec

In [ ]:
# Создадим еще одну функцию которая будет распознавать тестовую выборку и выводить результаты
def recognizerMultiClass (model, xTest, modelName):
  #print ("Название нейронной сети: ", modelName)        # Для наглядности выведем название сети
  #print ()                                              # Пропустим стороку
  totalSumRec = 0                                       # Создадим счетчик суммы правильных ответов

  # Пройдемся циклом по всем классам
  for i in range(nClasses):                             # От 0 до 6 
    currPred = model.predict(xTest[i])                  # Получим результаты распознования класса по блокам слов длинны xLen.
    currOut = np.argmax(currPred, axis=1)               # Определим номер распознанного класса с помощью функции argmax
    evVal = []
    
    for j in range(nClasses):
      evVal.append(len(currOut[currOut==j])/len(xTest[i]))

    totalSumRec +=len(currOut[currOut==i])
    recognizedClass = np.argmax(evVal)                  # Определяем какой класс за какой был распознан в итоге

    # И выводим результаты

    isRecognized = "Ответ НЕВЕРНЫЙ, УВЫ"
    if (recognizedClass == i):
      isRecognized = "УРА, ответ ВЕРНЫЙ!"
    
    str1 = 'Класс: ' + className[i] + " " * (11 - len(className[i])) + str(int(100*evVal[i])) + "% сеть предсказала: " + className[recognizedClass]
    print(str1, " " * (55-len(str1)), isRecognized, sep='')

  # Выведим средний процент распознавания 
  print ()                                                                          # Пустая строка чтобы отделить
  sumCount = 0
  # Пройдемся циклом по всем циклам
  for i in range(nClasses):
    sumCount += len(xTest[i])
  print ("Средний процент распознавания ", int(100*totalSumRec/sumCount), "%")
  print ()                                                                          # Пустая строка чтобы отделить

  return totalSumRec/sumCount

# Подготовка данных

In [ ]:
# Объявим классы. Запишем в список
className = ["О. Генри", "Стругацкие", "Булгаков", "Саймак", "Фрай", "Брэдберри"]

# Подсчитаем кол-во нужных нам классов
nClasses = len(className)

# Создадим 2 пустых списка (обучающий и проверочный), для последующего заполнения
trainText = []
testText = []

In [ ]:
# Пройдемся двумя циклами для заполнения списков. Так чтобы каждый i элемент в обучающей выборке соответствовал j элементу в тестовой.

for i in className:
  for j in os.listdir('/content/'):                     
    if i in j:                                       
      if 'Обучающая' in j:                           
        trainText.append(readText('/content/' +j))      
        print (j, 'добавлен в обучающую выборку')    
      if 'Тестовая' in j:                            
        testText.append(readText('/content/' +j))       
        print (j, 'добавлен в тестовую выборку')     

(О. Генри) Обучающая_50 вместе.txt добавлен в обучающую выборку
(О. Генри) Тестовая_20 вместе.txt добавлен в тестовую выборку
(Стругацкие) Тестовая_2 вместе.txt добавлен в тестовую выборку
(Стругацкие) Обучающая_5 вместе.txt добавлен в обучающую выборку
(Булгаков) Тестовая_2 вместе.txt добавлен в тестовую выборку
(Булгаков) Обучающая_5 вместе.txt добавлен в обучающую выборку
(Клиффорд_Саймак) Обучающая_5 вместе.txt добавлен в обучающую выборку
(Клиффорд_Саймак) Тестовая_2 вместе.txt добавлен в тестовую выборку
(Макс Фрай) Обучающая_5 вместе.txt добавлен в обучающую выборку
(Макс Фрай) Тестовая_2 вместе.txt добавлен в тестовую выборку
(Рэй Брэдберри) Обучающая_22 вместе.txt добавлен в обучающую выборку
(Рэй Брэдберри) Тестовая_8 вместе.txt добавлен в тестовую выборку


In [ ]:
# Создадим таблицу [Значения maxWordsCount, Число нейроново в слое Dense, Значение в слое Dropout, Активационная функция в первом слое Dense, Кол-во эпох, размер batch_size, Ошибки, Точность ]
df = pd.DataFrame(columns = ['Модель','Кол-во слоев','maxWordsCount','Dense', 'Dropout', 'activation', 'epochs', 'batch_size', 'learn_loss', 'learn_accuracy', 'val_loss', 'val_accuracy', 'Средний процент распознавания на проверочной выборке'])

count = 0            # счетчик для строк таблицы pandas

In [ ]:
# Зададим значения которые будут неизменными во всем ноутбуке.

xLen = 1000                                  # Длина отрезка текста на которые будет разбивать функция getSetFromIndexes
step = 100                                   # Шаг разбивания для той же функции
epochs = 15                                  # Количество эпох
batch_size = 150                             # Размер batch_size

# Задание А. Запустите нейронку c bag of words при разных maxWordsCount = 100, 1000, 10000, 50000

В этом задании меняется лишь один параметр, а все остальное остается одно и тоже. 

Чтобы понимать как изменения значения maxWordsCount влияет на обучаемость сети, остальные гиперпараметры нейронной сети изменяться не будут.

Для того чтобы 4 раза не переписывать подготовительные данные и нейронную сеть создадим цикл. Который будет перебирать значения maxWordsCount и в конце предоставит сравнительную таблицу.

In [ ]:
# Зададим параметры неронной сети. Чтобы проходить в цикле и использовать в pandas таблице.
# При желании можно быстро менять здесь значения и смотреть на результат обучения. 

maxWordsCount = [100, 1000, 10000, 25000]    # Создадим список чтобы проходиться по нему в цикле

dense_num = 300                              # Число нейронов в первом Dense слое
drop_num = 0.4                               # Значение в слое Dropout
activ = 'relu'                               # Активационная функция в первом слое Dense

hidden_layers = 2                            # Количество скрытых слоев, переменная нужна для записи в таблицу pandas

Параметр **verbose** установлен в **0**. Чтобы не было длинного вывода. Так как важны только крайние значения, которые будут предоставлены в сравнительной таблице.

При установке значения **maxWordsCount** равным 50000, 40000, 30000 происходит переполнение памяти в ноутбуке. Придется проведить тест с крайним значениев в **25000**.

In [ ]:
for i in maxWordsCount:

  
  tokenizer = Tokenizer(num_words = i, filters = '!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token ='unknow', char_level = False)
  tokenizer.fit_on_texts(trainText)   
  items = list(tokenizer.word_index.items())

  # Согласно частотному словарю можно преобразовать текст в последовательность индексов
  trainWordIndexes = tokenizer.texts_to_sequences(trainText)
  testWordIndexes = tokenizer.texts_to_sequences(testText)   

  # Установим базовые параметры

  # Сформируем выборки (обучающую и тестовую)
  xTrain, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
  xTest, yTest = createSetsMultiClasses(testWordIndexes, xLen, step)

  # Преобразовываем в матрицы нулей и единиц по принципу BOW. xTrain и yTrain подаем в виду списка. 
  xTrainBOW = tokenizer.sequences_to_matrix(xTrain.tolist())      
  xTestBOW = tokenizer.sequences_to_matrix(xTest.tolist()) 

  # Создадим полносвязную сеть
  modelBow = Sequential()
  modelBow.add(Dense(dense_num, input_dim=i, activation = activ))   
  modelBow.add(Dropout(drop_num))
  modelBow.add(BatchNormalization())
  modelBow.add(Dense(6, activation ='softmax'))

  # Скомпилируем ее
  modelBow.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

  # Обучаем сеть на выборке, xTrainBOW
  history = modelBow.fit(xTrainBOW, yTrain, epochs=epochs, batch_size=batch_size, validation_data=(xTestBOW, yTest), verbose=0)

  # Создаем пустой словарь. 
  dct = []  
  # В цикле проходимся по всем значениям из history                                                  
  for v in history.history.values():                                        
    dct.append (v)                                            # Результат выгружаются в словарь.                                                                
  result = np.array(dct)                                      # Переводим в numpy массив.

  # Распознавание проверочной выборки

  # Преобразуем тестовую выборку
  xTest6ClassesBow, x2 = createTestMultiClasses(testWordIndexes, xLen, step)

  # Проверим точность сети на BOW
  pred = recognizerMultiClass (modelBow, xTest6ClassesBow, "BOW: Dense-DropOut-BatchNormalization-Dense")

  # Формируем строчку в DataFrame.
  # В строке будет: Значения maxWordsCount, кол-во нейронов в Dense слое, размер batch_size, loss и accuracy во всех трех выборках 

  df.loc[count] = ['Bag of Words', hidden_layers, i, dense_num, drop_num , activ, epochs, batch_size, round(result[0][-1],2), round(result[1][-1]*100,2), round(result[2][-1],2), round(result[3][-1]*100,2), round(pred*100,2)]
  count += 1

  # Сохраним веса
  modelBow.save_weights(f'modelBow[{i}] - loss: {round(result[0][-1],2)} - accuracy: {round(result[1][-1]*100,2)} - val_loss: {round(result[2][-1],2)} - val_accuracy: {round(pred*100,2)}.h5')

In [ ]:
 df

,Модель,Кол-во слоев,maxWordsCount,Dense,Dropout,activation,epochs,batch_size,learn_loss,learn_accuracy,val_loss,val_accuracy,Средний процент распознавания на проверочной выборке
0,Bag of Words,2,100,300,0.4,relu,15,150,0.37,86.58,1.76,51.11,51.11
1,Bag of Words,2,1000,300,0.4,relu,15,150,0.00,100.00,0.50,84.10,84.10
2,Bag of Words,2,10000,300,0.4,relu,15,150,0.00,100.00,0.34,88.02,88.02
3,Bag of Words,2,25000,300,0.4,relu,15,150,0.00,100.00,0.29,90.65,90.65


# Б. Запустите нейронку c bag of words (01) при maxWordsCount = 20000 и разных архитектурах

1. Поменяйте количество нейронов в слоях
2. Поменяйте количество слоев
3. Поменяйте активационные функции слоев

In [ ]:
maxWordsCount =  20000                          # Задаем значение в 20000, как требуется в задании Б.

## Б. 1. Поменяйте количество нейронов в слоях

В данной задаче будет 2 цикла. Так как менять значения нейронов будем в первом Dense слое, а так же кол-во нейронов, которые будут отключаться в слое Dropout.

Всего будет 36 вариантов.

In [ ]:
dense_num = [10, 50, 100, 200, 600, 1000]       # Список нейронов в первом слое Dense
drop_num =  [0.1, 0.15, 0.2, 0.25, 0.35, 0.4]   # Список значение в слое Dropout
activ = 'relu'                                  # Активационная функция в первом слое Dense

hidden_layers = 2                               # Количество скрытых слоев, переменная нужна для записи в таблицу pandas

In [ ]:
for i in dense_num:
  for j in drop_num:
    tokenizer = Tokenizer(num_words = i, filters = '!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token ='unknow', char_level = False)
    tokenizer.fit_on_texts(trainText)   
    items = list(tokenizer.word_index.items())

    trainWordIndexes = tokenizer.texts_to_sequences(trainText)
    testWordIndexes = tokenizer.texts_to_sequences(testText)   

    # Установим базовые параметры

    # Сформируем выборки (обучающую и тестовую)
    xTrain, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
    xTest, yTest = createSetsMultiClasses(testWordIndexes, xLen, step)

    # Преобразовываем в матрицы нулей и единиц по принципу BOW. xTrain и yTrain подаем в виду списка. 
    xTrainBOW = tokenizer.sequences_to_matrix(xTrain.tolist())      
    xTestBOW = tokenizer.sequences_to_matrix(xTest.tolist()) 

    # Создадим полносвязную сеть
    modelBow = Sequential()
    modelBow.add(Dense(i, input_dim=i, activation = activ))   
    modelBow.add(Dropout(j))
    modelBow.add(BatchNormalization())
    modelBow.add(Dense(6, activation ='softmax'))

    # Скомпилируем ее
    modelBow.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Обучаем сеть на выборке, xTrainBOW
    history = modelBow.fit(xTrainBOW, yTrain, epochs=epochs, batch_size=batch_size, validation_data=(xTestBOW, yTest), verbose=0)

    # Создаем пустой словарь. 
    dct = []  
    # В цикле проходимся по всем значениям из history                                                  
    for v in history.history.values():                                        
      dct.append (v)                                            # Результат выгружаются в словарь.                                                                
    result = np.array(dct)                                      # Переводим в numpy массив.

    # Распознавание проверочной выборки

    # Преобразуем тестовую выборку
    xTest6ClassesBow, x2 = createTestMultiClasses(testWordIndexes, xLen, step)

    # Проверим точность сети на BOW
    pred = recognizerMultiClass (modelBow, xTest6ClassesBow, "BOW: Dense-DropOut-BatchNormalization-Dense")

    # Формируем строчку в DataFrame.
    # В строке будет: Значения maxWordsCount, кол-во нейронов в Dense слое, размер batch_size, loss и accuracy во всех трех выборках 

    df.loc[count] = ['Bag of Words', hidden_layers, maxWordsCount, i, j , activ, epochs, batch_size, round(result[0][-1],2), round(result[1][-1]*100,2), round(result[2][-1],2), round(result[3][-1]*100,2), round(pred*100,2)]
    count += 1

    # Сохраним веса
    modelBow.save_weights(f'modelBow[{i,j}] - loss: {round(result[0][-1],2)} - accuracy: {round(result[1][-1]*100,2)} - val_loss: {round(result[2][-1],2)} - val_accuracy: {round(pred*100,2)}.h5')

In [ ]:
# Посмотрим на часть таблицы. Так как она стала длиннее
df[-5:] 

,Модель,Кол-во слоев,maxWordsCount,Dense,Dropout,activation,epochs,batch_size,learn_loss,learn_accuracy,val_loss,val_accuracy,Средний процент распознавания на проверочной выборке
38,Bag of Words,2,20000,1000,0.15,relu,15,150,0.0,100.0,0.56,84.28,84.28
39,Bag of Words,2,20000,1000,0.2,relu,15,150,0.0,100.0,0.51,84.92,84.92
40,Bag of Words,2,20000,1000,0.25,relu,15,150,0.0,100.0,0.53,84.55,84.55
41,Bag of Words,2,20000,1000,0.35,relu,15,150,0.0,100.0,0.52,84.19,84.19
42,Bag of Words,2,20000,1000,0.4,relu,15,150,0.0,100.0,0.49,85.30,85.30


## Б. 2. Поменяйте количество слоев

Добавим еще скрытых слоев. Посмотрим на результаты предыдущих обучений, чтобы установить лучшие гиперпараметры. Протестируем сеть и добавим данные в таблицу.
 
По прежнему maxWordsCount = 20000.

In [ ]:
dense_num_1 = 300                                # Кол-во нейронов в первом слое Dense
dense_num_2 = 600                                # Кол-во нейронов во втором слое Dense

drop_num_1 =  0.3                               # Значение в первом  Dropout слое
drop_num_2 =  0.4                               # Значение во втором  Dropout слое

activ = 'relu'                                  # Активационная функция в первом слое Dense

hidden_layers = 5                               # Количество скрытых слоев, переменная нужна для записи в таблицу pandas

In [ ]:
tokenizer = Tokenizer(num_words = maxWordsCount, filters = '!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token ='unknow', char_level = False)
tokenizer.fit_on_texts(trainText)   
items = list(tokenizer.word_index.items())

trainWordIndexes = tokenizer.texts_to_sequences(trainText)
testWordIndexes = tokenizer.texts_to_sequences(testText)   

# Сформируем выборки (обучающую и тестовую)
xTrain, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
xTest, yTest = createSetsMultiClasses(testWordIndexes, xLen, step)

# Преобразовываем в матрицы нулей и единиц по принципу BOW. xTrain и yTrain подаем в виду списка. 
xTrainBOW = tokenizer.sequences_to_matrix(xTrain.tolist())      
xTestBOW = tokenizer.sequences_to_matrix(xTest.tolist()) 

# Создадим полносвязную сеть
modelBow = Sequential()

modelBow.add(Dense(dense_num_1, input_dim=maxWordsCount, activation = activ))   
modelBow.add(Dropout(drop_num_1))
modelBow.add(BatchNormalization())

modelBow.add(Dense(dense_num_2, input_dim=maxWordsCount, activation = activ))   
modelBow.add(Dropout(drop_num_2))
modelBow.add(BatchNormalization())

modelBow.add(Dense(6, activation ='softmax'))

# Скомпилируем ее
modelBow.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучаем сеть на выборке, xTrainBOW
history = modelBow.fit(xTrainBOW, yTrain, epochs=epochs, batch_size=batch_size, validation_data=(xTestBOW, yTest), verbose=0)

# Создаем пустой словарь. 
dct = []  
# В цикле проходимся по всем значениям из history                                                  
for v in history.history.values():                                        
  dct.append (v)                                            # Результат выгружаются в словарь.                                                                
result = np.array(dct)                                      # Переводим в numpy массив.

# Распознавание проверочной выборки

# Преобразуем тестовую выборку
xTest6ClassesBow, x2 = createTestMultiClasses(testWordIndexes, xLen, step)

# Проверим точность сети на BOW
pred = recognizerMultiClass (modelBow, xTest6ClassesBow, "BOW: Dense-DropOut-BatchNormalization-Dense")

# Формируем строчку в DataFrame.
# В строке будет: Значения maxWordsCount, кол-во нейронов в Dense слое, размер batch_size, loss и accuracy во всех трех выборках 

df.loc[count] = ['Bag of Words', hidden_layers, i, f'{dense_num_1}-{dense_num_2}', f'{drop_num_1}-{drop_num_2}' , activ, epochs, batch_size, round(result[0][-1],2), round(result[1][-1]*100,2), round(result[2][-1],2), round(result[3][-1]*100,2), round(pred*100,2)]
count += 1

# Сохраним веса
modelBow.save_weights(f'modelBow[{i}] - loss: {round(result[0][-1],2)} - accuracy: {round(result[1][-1]*100,2)} - val_loss: {round(result[2][-1],2)} - val_accuracy: {round(pred*100,2)}.h5')

In [ ]:
df[-5:]

,Модель,Кол-во слоев,maxWordsCount,Dense,Dropout,activation,epochs,batch_size,learn_loss,learn_accuracy,val_loss,val_accuracy,Средний процент распознавания на проверочной выборке
39,Bag of Words,2,20000,1000,0.2,relu,15,150,0.0,100.0,0.51,84.92,84.92
40,Bag of Words,2,20000,1000,0.25,relu,15,150,0.0,100.0,0.53,84.55,84.55
41,Bag of Words,2,20000,1000,0.35,relu,15,150,0.0,100.0,0.52,84.19,84.19
42,Bag of Words,2,20000,1000,0.4,relu,15,150,0.0,100.0,0.49,85.30,85.30
43,Bag of Words,5,1000,300-600,0.3-0.4,relu,15,150,0.0,100.0,0.36,88.60,88.60


**Добавим еще скрытых слоев**.



In [ ]:
dense_num_1 = 300                                # Кол-во нейронов в первом слое Dense
dense_num_2 = 600                                # Кол-во нейронов во втором слое Dense
dense_num_3 = 900                                # Кол-во нейронов во втором слое Dense

drop_num_1 =  0.2                                # Значение в первом  Dropout слое
drop_num_2 =  0.3                                # Значение во втором  Dropout слое
drop_num_3 =  0.4                                # Значение во втором  Dropout слое

activ = 'relu'                                   # Активационная функция в первом слое Dense

layers = 8                                       # Количество скрытый слоев, переменная нужна для таблицы

In [ ]:
tokenizer = Tokenizer(num_words = maxWordsCount, filters = '!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token ='unknow', char_level = False)
tokenizer.fit_on_texts(trainText)   
items = list(tokenizer.word_index.items())

trainWordIndexes = tokenizer.texts_to_sequences(trainText)
testWordIndexes = tokenizer.texts_to_sequences(testText)   

# Установим базовые параметры

# Сформируем выборки (обучающую и тестовую)
xTrain, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
xTest, yTest = createSetsMultiClasses(testWordIndexes, xLen, step)

# Преобразовываем в матрицы нулей и единиц по принципу BOW. xTrain и yTrain подаем в виду списка. 
xTrainBOW = tokenizer.sequences_to_matrix(xTrain.tolist())      
xTestBOW = tokenizer.sequences_to_matrix(xTest.tolist()) 

# Создадим полносвязную сеть
modelBow = Sequential()
modelBow.add(Dense(dense_num_1, input_dim=maxWordsCount, activation = activ))   
modelBow.add(Dropout(drop_num_1))
modelBow.add(BatchNormalization())

modelBow.add(Dense(dense_num_2, input_dim=i, activation = activ))   
modelBow.add(Dropout(drop_num_2))
modelBow.add(BatchNormalization())

modelBow.add(Dense(dense_num_3, input_dim=i, activation = activ))   
modelBow.add(Dropout(drop_num_3))
modelBow.add(BatchNormalization())

modelBow.add(Dense(6, activation ='softmax'))

# Скомпилируем ее
modelBow.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучаем сеть на выборке, xTrainBOW
history = modelBow.fit(xTrainBOW, yTrain, epochs=epochs, batch_size=batch_size, validation_data=(xTestBOW, yTest), verbose=0)

# Создаем пустой словарь. 
dct = []  
# В цикле проходимся по всем значениям из history                                                  
for v in history.history.values():                                        
  dct.append (v)                                            # Результат выгружаются в словарь.                                                                
result = np.array(dct)                                      # Переводим в numpy массив.

# Распознавание проверочной выборки

# Преобразуем тестовую выборку
xTest6ClassesBow, x2 = createTestMultiClasses(testWordIndexes, xLen, step)

# Проверим точность сети на BOW
pred = recognizerMultiClass (modelBow, xTest6ClassesBow, "BOW: Dense-DropOut-BatchNormalization-Dense")

# Формируем строчку в DataFrame.

df.loc[count] = ['Bag of Words', hidden_layers, i, f'{dense_num_1}-{dense_num_2}-{dense_num_3}', f'{drop_num_1}-{drop_num_2}-{drop_num_3}' , activ, epochs, batch_size, round(result[0][-1],2), round(result[1][-1]*100,2), round(result[2][-1],2), round(result[3][-1]*100,2), round(pred*100,2)]
count += 1


# Сохраним веса
modelBow.save_weights(f'modelBow[layers = 5] - loss: {round(result[0][-1],2)} - accuracy: {round(result[1][-1]*100,2)} - val_loss: {round(result[2][-1],2)} - val_accuracy: {round(pred*100,2)}.h5')

In [ ]:
df[-5:]

,Модель,Кол-во слоев,maxWordsCount,Dense,Dropout,activation,epochs,batch_size,learn_loss,learn_accuracy,val_loss,val_accuracy,Средний процент распознавания на проверочной выборке
40,Bag of Words,2,20000,1000,0.25,relu,15,150,0.0,100.0,0.53,84.55,84.55
41,Bag of Words,2,20000,1000,0.35,relu,15,150,0.0,100.0,0.52,84.19,84.19
42,Bag of Words,2,20000,1000,0.4,relu,15,150,0.0,100.0,0.49,85.30,85.30
43,Bag of Words,5,1000,300-600,0.3-0.4,relu,15,150,0.0,100.0,0.36,88.60,88.60
44,Bag of Words,5,1000,300-600-900,0.2-0.3-0.4,relu,15,150,0.0,100.0,0.42,87.27,87.27


## Б. 3. Поменяйте активационные функции слоев

Для первого примера в этом задании возьмем изначальную сеть с 2мя скрытыми слоями и будем менять активационые функции в первом Dense слое используя цикл. 

Остальные гиперпараметры сделаем такими же, чтобы посмотреть как повлияет только изменение активационной функции:

- Кол-во нейронов в Dense слое 300
- Dropout 0.4
- Была функция 'relu'. Будем перебирать 'linear', 'softmax', 'sigmoid', 'softplus', 'tanh'.


In [ ]:
dense_num_1 = 300                                             # Кол-во нейронов в первом слое Dense
drop_num_1 = 0.4                                              # Значение в слое Dropout

activ = ['linear','softmax','softplus', 'sigmoid','tanh']     # Активационная функция в первом слое Dense  'softmax', 'linear','sigmoid',

hidden_layers = 2                                             # Количество скрытый слоев, переменная нужна для таблицы

In [ ]:
for i in activ:

  tokenizer = Tokenizer(num_words = maxWordsCount, filters = '!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token ='unknow', char_level = False)
  tokenizer.fit_on_texts(trainText)   
  items = list(tokenizer.word_index.items())

  trainWordIndexes = tokenizer.texts_to_sequences(trainText)
  testWordIndexes = tokenizer.texts_to_sequences(testText)   

  # Установим базовые параметры

  # Сформируем выборки (обучающую и тестовую)
  xTrain, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
  xTest, yTest = createSetsMultiClasses(testWordIndexes, xLen, step)

  # Преобразовываем в матрицы нулей и единиц по принципу BOW. xTrain и yTrain подаем в виду списка. 
  xTrainBOW = tokenizer.sequences_to_matrix(xTrain.tolist())      
  xTestBOW = tokenizer.sequences_to_matrix(xTest.tolist()) 

  # Создадим полносвязную сеть
  modelBow = Sequential()
  modelBow.add(Dense(dense_num_1, input_dim=maxWordsCount, activation = i))   
  modelBow.add(Dropout(drop_num_1))
  modelBow.add(BatchNormalization())
  modelBow.add(Dense(6, activation ='softmax'))

  # Скомпилируем ее
  modelBow.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

  # Обучаем сеть на выборке, xTrainBOW
  history = modelBow.fit(xTrainBOW, yTrain, epochs=epochs, batch_size=batch_size, validation_data=(xTestBOW, yTest), verbose=0)

  # Создаем пустой словарь. 
  dct = []  
  # В цикле проходимся по всем значениям из history                                                  
  for v in history.history.values():                                        
    dct.append (v)                                            # Результат выгружаются в словарь.                                                                
  result = np.array(dct)                                      # Переводим в numpy массив.

  # Распознавание проверочной выборки

  # Преобразуем тестовую выборку
  xTest6ClassesBow, x2 = createTestMultiClasses(testWordIndexes, xLen, step)

  # Проверим точность сети на BOW
  pred = recognizerMultiClass (modelBow, xTest6ClassesBow, "BOW: Dense-DropOut-BatchNormalization-Dense")

  # Формируем строчку в DataFrame.
  # В строке будет: Значения maxWordsCount, кол-во нейронов в Dense слое, размер batch_size, loss и accuracy во всех трех выборках 

  df.loc[count] = ['Bag of Words', hidden_layers, maxWordsCount, dense_num_1, drop_num_1 , i, epochs, batch_size, round(result[0][-1],2), round(result[1][-1]*100,2), round(result[2][-1],2), round(result[3][-1]*100,2), round(pred*100,2)]
  count += 1

  # Сохраним веса
  modelBow.save_weights(f'modelBow[{i}] - loss: {round(result[0][-1],2)} - accuracy: {round(result[1][-1]*100,2)} - val_loss: {round(result[2][-1],2)} - val_accuracy: {round(pred*100,2)}.h5')

In [ ]:
df[-5:]

,Модель,Кол-во слоев,maxWordsCount,Dense,Dropout,activation,epochs,batch_size,learn_loss,learn_accuracy,val_loss,val_accuracy,Средний процент распознавания на проверочной выборке
45,Bag of Words,2,20000,300,0.4,linear,15,150,0.00,100.00,0.35,88.45,88.45
46,Bag of Words,2,20000,300,0.4,softmax,15,150,0.09,97.42,0.51,83.11,83.11
47,Bag of Words,2,20000,300,0.4,softplus,15,150,0.00,100.00,0.28,90.68,90.68
48,Bag of Words,2,20000,300,0.4,sigmoid,15,150,0.00,100.00,0.27,92.22,92.22
49,Bag of Words,2,20000,300,0.4,tanh,15,150,0.00,100.00,0.34,88.59,88.59


**Проверим какие показатели будут у сети, если добавить еще слоев как предыдущем задании и поменять активационные функции**. 

In [ ]:
dense_num_1 = 300                                            # Кол-во нейронов в первом слое Dense
dense_num_2 = 600                                            # Кол-во нейронов во втором слое Dense

drop_num_1 =  0.3                                            # Значение в первом  Dropout слое
drop_num_2 =  0.4                                            # Значение во втором  Dropout слое

activ = ['linear','softmax','softplus', 'sigmoid','tanh']    # Активационная функция в первом слое Dense
                                               
hidden_layers = 5                                            # Количество скрытых слоев, переменная нужна для записи в таблицу pandas

In [ ]:
for i in activ:

  tokenizer = Tokenizer(num_words = maxWordsCount, filters = '!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token ='unknow', char_level = False)
  tokenizer.fit_on_texts(trainText)   
  items = list(tokenizer.word_index.items())

  trainWordIndexes = tokenizer.texts_to_sequences(trainText)
  testWordIndexes = tokenizer.texts_to_sequences(testText)   

  # Установим базовые параметры

  # Сформируем выборки (обучающую и тестовую)
  xTrain, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
  xTest, yTest = createSetsMultiClasses(testWordIndexes, xLen, step)

  # Преобразовываем в матрицы нулей и единиц по принципу BOW. xTrain и yTrain подаем в виду списка. 
  xTrainBOW = tokenizer.sequences_to_matrix(xTrain.tolist())      
  xTestBOW = tokenizer.sequences_to_matrix(xTest.tolist()) 

  # Создадим полносвязную сеть
  modelBow = Sequential()

  modelBow.add(Dense(dense_num_1, input_dim=maxWordsCount, activation = i))   
  modelBow.add(Dropout(drop_num_1))
  modelBow.add(BatchNormalization())

  modelBow.add(Dense(dense_num_2, input_dim=maxWordsCount, activation = i))   
  modelBow.add(Dropout(drop_num_2))
  modelBow.add(BatchNormalization())

  modelBow.add(Dense(6, activation ='softmax'))

  # Скомпилируем ее
  modelBow.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

  # Обучаем сеть на выборке, xTrainBOW
  history = modelBow.fit(xTrainBOW, yTrain, epochs=epochs, batch_size=batch_size, validation_data=(xTestBOW, yTest), verbose=0)

  # Создаем пустой словарь. 
  dct = []  
  # В цикле проходимся по всем значениям из history                                                  
  for v in history.history.values():                                        
    dct.append (v)                                            # Результат выгружаются в словарь.                                                                
  result = np.array(dct)                                      # Переводим в numpy массив.

  # Распознавание проверочной выборки

  # Преобразуем тестовую выборку
  xTest6ClassesBow, x2 = createTestMultiClasses(testWordIndexes, xLen, step)

  # Проверим точность сети на BOW
  pred = recognizerMultiClass (modelBow, xTest6ClassesBow, "BOW: Dense-DropOut-BatchNormalization-Dense")

  df.loc[count] = ['Bag of Words', hidden_layers, maxWordsCount, f'{dense_num_1}-{dense_num_2}', f'{drop_num_1}-{drop_num_2}', i, epochs, batch_size, round(result[0][-1],2), round(result[1][-1]*100,2), round(result[2][-1],2), round(result[3][-1]*100,2), round(pred*100,2)]
  count += 1

  # Сохраним веса
  modelBow.save_weights(f'modelBow[{i}] - loss: {round(result[0][-1],2)} - accuracy: {round(result[1][-1]*100,2)} - val_loss: {round(result[2][-1],2)} - val_accuracy: {round(pred*100,2)}.h5')

In [ ]:
df[-5:]

,Модель,Кол-во слоев,maxWordsCount,Dense,Dropout,activation,epochs,batch_size,learn_loss,learn_accuracy,val_loss,val_accuracy,Средний процент распознавания на проверочной выборке
50,Bag of Words,5,20000,300-600,0.3-0.4,linear,15,150,0.00,100.00,0.29,90.31,90.31
51,Bag of Words,5,20000,300-600,0.3-0.4,softmax,15,150,0.08,97.02,1.85,71.31,71.31
52,Bag of Words,5,20000,300-600,0.3-0.4,softplus,15,150,0.00,100.00,0.27,91.38,91.38
53,Bag of Words,5,20000,300-600,0.3-0.4,sigmoid,15,150,0.00,100.00,0.29,90.62,90.62
54,Bag of Words,5,20000,300-600,0.3-0.4,tanh,15,150,0.00,100.00,0.39,88.77,88.77


# В. Запустите нейронку c Embbedding при maxWordsCount = 50000, поменяйте размер Embedding пространства:
# 1. 10
# 2. 50
# 3. 200

## Данные параметры будем перебирать в цикле, чтобы не было много строчек кода. 
   Параметр maxWordsCount пришлось установить равным **25000**. Если поставить **50000**, то идет переполнение памяти и ноутбук перезагружается.  

In [ ]:
emb_space = [10,50,200]                         # Создадим список для изменения Embbedding пространства

maxWordsCount =  25000                        
           
dense_num_1 = 500                               # Кол-во нейронов в первом слое Dense
spat_drop_out = 0.2                             # Значение в слое SpatialDropout1D
drop_out_1 =  0.3                               # Значение в первом  Dropout слое

activ = 'relu'                                  # Активационная функция в первом слое Dense
                                                
hidden_layers = 6                               # Количество скрытых слоев, переменная нужна для записи в таблицу pandas

In [ ]:
for i in emb_space:

  tokenizer = Tokenizer(num_words = maxWordsCount, filters = '!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token ='unknow', char_level = False)
  tokenizer.fit_on_texts(trainText)   
  items = list(tokenizer.word_index.items())

  trainWordIndexes = tokenizer.texts_to_sequences(trainText)
  testWordIndexes = tokenizer.texts_to_sequences(testText)   

  # Установим базовые параметры

  # Сформируем выборки (обучающую и тестовую)
  xTrain, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
  xTest, yTest = createSetsMultiClasses(testWordIndexes, xLen, step)

  # Преобразовываем в матрицы нулей и единиц по принципу BOW. xTrain и yTrain подаем в виду списка. 
  xTrainBOW = tokenizer.sequences_to_matrix(xTrain.tolist())      
  xTestBOW = tokenizer.sequences_to_matrix(xTest.tolist()) 


  # Создаем нейронную сеть со слоем Embedding  и activation='softmax'
  modelEmb = Sequential()

  modelEmb.add(Embedding(maxWordsCount, i, input_length=xLen))       # maxWordsCount = 30000
  modelEmb.add(SpatialDropout1D(spat_drop_out))

  modelEmb.add(Flatten())
  modelEmb.add(BatchNormalization())

  modelEmb.add(Dense(dense_num_1, activation = activ))
  modelEmb.add(Dropout(drop_out_1))
  modelEmb.add(BatchNormalization())

  modelEmb.add(Dense(6,activation='softmax'))

  # Скомпилируем данную сеть
  modelEmb.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

  # Обучим сеть на обучающей выборке xTrain
  history = modelEmb.fit(xTrain, yTrain, epochs=1, batch_size=150, validation_data=(xTest, yTest), verbose=0)

  # Создаем пустой словарь. 
  dct = []  
  # В цикле проходимся по всем значениям из history                                                  
  for v in history.history.values():                                        
    dct.append (v)                                            # Результат выгружаются в словарь.                                                                
  result = np.array(dct)                                      # Переводим в numpy массив.

  # Проверим результаты Embedding сети

  # Преобразуем тестовую выборку
  _, xtest6Classes = createTestMultiClasses(testWordIndexes, xLen, step)

  pred = recognizerMultiClass (modelEmb, xtest6Classes, "Embedding")

  df.loc[count] = [f'Embbedding - {i}', hidden_layers, maxWordsCount, dense_num_1, f'{spat_drop_out }-{drop_out_1}', activ, epochs, batch_size, round(result[0][-1],2), round(result[1][-1]*100,2), round(result[2][-1],2), round(result[3][-1]*100,2), round(pred*100,2)]
  count += 1

  # Сохраним веса
  modelEmb.save_weights(f'Embbedding[{i}] - loss: {round(result[0][-1],2)} - accuracy: {round(result[1][-1]*100,2)} - val_loss: {round(result[2][-1],2)} - val_accuracy: {round(pred*100,2)}.h5')

In [ ]:
df

,Модель,Кол-во слоев,maxWordsCount,Dense,Dropout,activation,epochs,batch_size,learn_loss,learn_accuracy,val_loss,val_accuracy,Средний процент распознавания на проверочной выборке
0,Embbedding - 10,6,25000,500,0.2-0.3,relu,15,150,1.06,66.88,1.59,30.08,30.08
1,Embbedding - 50,6,25000,500,0.2-0.3,relu,15,150,0.78,79.15,1.97,19.32,19.32
2,Embbedding - 200,6,25000,500,0.2-0.3,relu,15,150,0.73,82.56,1.76,39.72,39.72


# Так как таблица получилась слишком большая и найти в ней нужные нам данные будет сложно, посмотрим на топ 5 лучших и худших результатов. 

Последнее задание с Embbedding запускал в самом начале, так как на его исполнение требовалось больше памяти. И если запускать его в конце, то идет переполнение памяти и ноутбук перезагружается. 

In [ ]:
# Общая таблица
df

,Модель,Кол-во слоев,maxWordsCount,Dense,Dropout,activation,epochs,batch_size,learn_loss,learn_accuracy,val_loss,val_accuracy,Средний процент распознавания на проверочной выборке
0,Embbedding - 10,6,25000,500,0.2-0.3,relu,15,150,1.06,66.88,1.59,30.08,30.08
1,Embbedding - 50,6,25000,500,0.2-0.3,relu,15,150,0.78,79.15,1.97,19.32,19.32
2,Embbedding - 200,6,25000,500,0.2-0.3,relu,15,150,0.73,82.56,1.76,39.72,39.72
3,Bag of Words,2,100,300,0.4,relu,15,150,0.37,86.67,1.92,49.90,49.90
4,Bag of Words,2,1000,300,0.4,relu,15,150,0.00,100.00,0.47,85.15,85.15
5,Bag of Words,2,10000,300,0.4,relu,15,150,0.00,100.00,0.32,88.44,88.44
6,Bag of Words,2,25000,300,0.4,relu,15,150,0.00,100.00,0.33,88.96,88.96
7,Bag of Words,2,20000,10,0.1,relu,15,150,1.67,33.93,1.76,29.51,29.51
8,Bag of Words,2,20000,10,0.15,relu,15,150,1.68,34.08,1.75,29.60,29.60
9,Bag of Words,2,20000,10,0.2,relu,15,150,1.68,33.78,1.74,29.60,29.60


# Так как таблица получилась очень большая из-за такого количества разных вариантов и искать в ней нужные для нас данные будет не просто, то выведем лучшие 5 результатов. 

In [ ]:
print('5 лучших результата по максимальному среднему проценту')
df.sort_values('Средний процент распознавания на проверочной выборке', ascending=False).head(5)

5 лучших результата по максимальному среднему проценту


,Модель,Кол-во слоев,maxWordsCount,Dense,Dropout,activation,epochs,batch_size,learn_loss,learn_accuracy,val_loss,val_accuracy,Средний процент распознавания на проверочной выборке
48,Bag of Words,2,20000,300,0.4,sigmoid,15,150,0.0,100.0,0.27,92.22,92.22
52,Bag of Words,5,20000,300-600,0.3-0.4,softplus,15,150,0.0,100.0,0.27,91.38,91.38
47,Bag of Words,2,20000,300,0.4,softplus,15,150,0.0,100.0,0.28,90.68,90.68
53,Bag of Words,5,20000,300-600,0.3-0.4,sigmoid,15,150,0.0,100.0,0.29,90.62,90.62
50,Bag of Words,5,20000,300-600,0.3-0.4,linear,15,150,0.0,100.0,0.29,90.31,90.31


In [ ]:
print('5 лучших результата по максимальной точности тестовой выборки')
df.sort_values('val_accuracy', ascending=False).head(5)

5 лучших результата по максимальной точности тестовой выборки


,Модель,Кол-во слоев,maxWordsCount,Dense,Dropout,activation,epochs,batch_size,learn_loss,learn_accuracy,val_loss,val_accuracy,Средний процент распознавания на проверочной выборке
48,Bag of Words,2,20000,300,0.4,sigmoid,15,150,0.0,100.0,0.27,92.22,92.22
52,Bag of Words,5,20000,300-600,0.3-0.4,softplus,15,150,0.0,100.0,0.27,91.38,91.38
47,Bag of Words,2,20000,300,0.4,softplus,15,150,0.0,100.0,0.28,90.68,90.68
53,Bag of Words,5,20000,300-600,0.3-0.4,sigmoid,15,150,0.0,100.0,0.29,90.62,90.62
50,Bag of Words,5,20000,300-600,0.3-0.4,linear,15,150,0.0,100.0,0.29,90.31,90.31


In [ ]:
print('5 лучших результата по минимальной ошибке тестовой выборки')
df.sort_values('val_loss').head(5)

5 лучших результата по минимальной ошибке тестовой выборки


,Модель,Кол-во слоев,maxWordsCount,Dense,Dropout,activation,epochs,batch_size,learn_loss,learn_accuracy,val_loss,val_accuracy,Средний процент распознавания на проверочной выборке
52,Bag of Words,5,20000,300-600,0.3-0.4,softplus,15,150,0.0,100.0,0.27,91.38,91.38
48,Bag of Words,2,20000,300,0.4,sigmoid,15,150,0.0,100.0,0.27,92.22,92.22
47,Bag of Words,2,20000,300,0.4,softplus,15,150,0.0,100.0,0.28,90.68,90.68
50,Bag of Words,5,20000,300-600,0.3-0.4,linear,15,150,0.0,100.0,0.29,90.31,90.31
53,Bag of Words,5,20000,300-600,0.3-0.4,sigmoid,15,150,0.0,100.0,0.29,90.62,90.62


# Выводы: 

1. В данных примерах прослеживается закономерность, что чем больше кол-во слов в словаре, тем большая точность на тестовой выборке.

    При значении maxWordsCount равным 20000 точность значительно выше чем при 100. Можно предположить что при значении в 50000 точность будет выше. 

2. В данном случаи, при такой архитектуре нейронных сетей и с одинаковыми параметрами метод Bag of Words показал лучше результат чем Embbedding. 

3. В сравнительно таблице видно, что кол-во нейронов в полносвязном Dense слое тоже влияют на результат. Так даже при большом словаре и 10 нейронов в Dense слое, показатели не удовлетворительные. 

4. Значение в слое Dropout тоже влияет. Так данные эксперименты показали что лучшие показатели при значении 0.4. 

5. Добавление новых слоев увеличивают средний процент на тестовой выборке, но не значительно.

6. Так же как и использование других активациооных функций. 


7. Но не смотря на это даже прирост в несколько процентов имееет смысл. Поэтому нужно эксперементировать и подбирать лучшую архитектуру и гиперпарметры нейронной сети. 